## Drone Detection YOLOv8


In this Project, we are going to Detect Drones in an Image

We are going to use YOLOv8

Important Links:
* Github YoloV8 ultralytics: https://github.com/ultralytics/ultralytics
* Ultralytics Object-detection Models: https://docs.ultralytics.com/models/  

In [ ]:
!pip install ultralytics
import ultralytics
from ultralytics import YOLO

import torch
import torchvision

import numpy
import PIL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 3.6 MB/s eta 0:00:00


### Mount Google drive
We have to mount Google drive, so that we can use our dataset

**Step 1:** Mount Google drive


**Step 2:** Find Dataset and Unzip it in Colab main directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/drone_dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

## YOLOv8: Inference mode

There are two methods to make inference - Command Line (CLI) and Python code

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

results = model("https://ultralytics.com/images/bus.jpg", show=True, save=True)  # predict on an image

100%|██████████| 6.23M/6.23M [00:00<00:00, 84.6MB/s]

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



100%|██████████| 476k/476k [00:00<00:00, 13.4MB/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 330.8ms
Speed: 13.3ms preprocess, 330.8ms inference, 2331.9ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict


In [ ]:
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg'

Ultralytics YOLOv8.0.235 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

100% 165k/165k [00:00<00:00, 8.13MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 198.9ms
Speed: 3.6ms preprocess, 198.9ms inference, 509.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


## YOLOv8 on custom Drone Detection Dataset

First step is to structure our data into a format that YOLO requires:
(For training)
* Collect Images and put them in directories - `data/train/images`
* Annotate these images: https://www.cvat.ai/ (and convert annotations to YOLO format)
* Download annotations to - `data/train/labels`
* Annotations should be in the respective format: label_num, x, y, w, h
Here Label Num is the label number the object in the image has, and x, y are th center points of the Bbox and w, h are the width and height respectively.

Next, We build up our configuration file as **config.yaml**

Example of config file:
```
# PATHS
path: /content/drone_dataset
train: train/images/
val: valid/images/

# Classes
names:
  0: drone

```

In [ ]:
%%writefile config.yaml

# PATHS
path: /content/drone_dataset
train: train/images/
val: valid/images/

# Classes
names:
  0: drone

Writing config.yaml


In [ ]:
from ultralytics import YOLO
import os

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Use the model
model.train(data=os.path.join('/content', "config.yaml"), epochs=100)  # train the model


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 755k/755k [00:00<00:00, 17.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drone_dataset/train/labels... 1012 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1012/1012 [00:00<00:00, 1167.57it/s]

train: WARNING ⚠️ /content/drone_dataset/train/images/pic_722.jpg: corrupt JPEG restored and saved
train: New cache created: /content/drone_dataset/train/labels.cache


...

                   all        347        369      0.878      0.756      0.838      0.473



100 epochs completed in 1.138 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.235 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:09<00:00,  1.14it/s]


                   all        347        369      0.861      0.748      0.841      0.493
Speed: 0.3ms preprocess, 2.9ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e4f46459cc0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 